In [1]:
import os
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential, ManagedIdentityCredential

In [2]:
AZURE_DIRECTORY = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/aimethica-general/code/Users/alexandru.popescu/anomaly_DOCUMENTE_FACTURI_EXPORT'

In [3]:
subscription_id = "9107822e-69ab-454f-a905-b59c4e15c1af"
resource_group = "AiMethica-ML"
workspace = "AiMethica-AZML"

In [4]:
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

In [5]:
ws = ml_client.workspaces.get(workspace)
print(ws.location, ":", ws.resource_group)

westeurope : AiMethica-ML


In [6]:
from azure.ai.ml.entities import AmlCompute

cpu_compute_target = "aimethica-cluster-managed"

try:
    cpu_cluster = ml_client.compute.get(cpu_compute_target)
    print(
        f"Cluster: {cpu_compute_target} exists."
    )

except Exception:
    print("The cluster doesn't exist.")

Cluster: aimethica-cluster-managed exists.


In [9]:
from azure.ai.ml import command
from azure.ai.ml import Input

custom_env_name = "aimethica-anomaly-env"
env_name = custom_env_name + "@latest"

registered_model_name = "anomaly_document_facturi_export"

code_directory = os.path.join(AZURE_DIRECTORY,"src")
input_data_train_path = os.path.join(AZURE_DIRECTORY,"data/DOCUMENTE_FACTURI_EXPORT_train.csv")
input_data_test_path = os.path.join(AZURE_DIRECTORY,"data/DOCUMENTE_FACTURI_EXPORT_test.csv")


job = command(
    inputs=dict(
        input_data_train=Input(
            type="uri_file",
            path=input_data_train_path,
        ),
        input_data_test=Input(
            type="uri_file",
            path=input_data_test_path,
        ),
        registered_model_name=registered_model_name,
        epochs = 50,
        batch_size = 16,
    ),
    compute=cpu_compute_target,
    code=code_directory,  # location of source code
    command="python anomaly_document_facturi_export.py --input_data_train ${{inputs.input_data_train}} --input_data_test ${{inputs.input_data_test}} --epochs ${{inputs.epochs}} --batch_size ${{inputs.batch_size}} --registered_model_name ${{inputs.registered_model_name}}",
    environment=env_name,
    display_name="anomaly_document_facturi_export"
)

In [10]:
returned_job = ml_client.create_or_update(job)
returned_job

Uploading src (0.01 MBs): 100%|██████████| 9652/9652 [00:00<00:00, 227196.27it/s]




Experiment,Name,Type,Status,Details Page
code,careful_map_52rd1875dq,command,Starting,Link to Azure Machine Learning studio
